In [ ]:
import pandas as pd
import geopandas as gpd
import folium
import googlemaps
from shapely.geometry import Point

# Substitua 'YOUR_API_KEY' pela sua chave real da API de Geocodificação do Google Maps
api_key = 'sua_chave_aqui'

# Carregue seus dados em um DataFrame
df_mun_func = pd.read_csv('E:/GitHub/UFU_MAPPERS/microdados/dados/microdados_uberlandia_em_funcionamento.csv', delimiter=';',
                           encoding='iso-8859-1', low_memory=False)

# Inicialize o cliente de Geocodificação do Google Maps
gmaps = googlemaps.Client(key=api_key)

In [ ]:
# Defina uma função para geocodificar um endereço usando a API de Geocodificação do Google Maps
def geocode_address(endereco):
    try:
        geocode_result = gmaps.geocode(endereco)
        if geocode_result:
            localizacao = geocode_result[0]['geometry']['location']
            print(localizacao)
            return Point(localizacao['lng'], localizacao['lat'])
        else:
            return None
    except Exception as e:
        print(f"Erro na geocodificação do endereço: {endereco}\nErro: {str(e)}")
        return None


In [ ]:
# Etapa 4: Crie uma nova coluna 'full_address' com o endereço combinado
df_mun_func['full_address'] = 'Brasil' + ', ' + 'MG' + ', ' + df_mun_func['NO_MUNICIPIO'] + ', ' + df_mun_func['DS_ENDERECO'] + ', ' + df_mun_func['NU_ENDERECO']

# Etapa 5: Aplique a função de geocodificação para criar uma coluna 'geometry'
df_mun_func['geometry'] = df_mun_func['full_address'].apply(geocode_address)

# Etapa 6: Crie um GeoDataFrame a partir do DataFrame
gdf = gpd.GeoDataFrame(df_mun_func, geometry='geometry')

# Etapa 6.a Salve o GeoDataFrame em um Shapefile
gdf.to_file('E:/GitHub/UFU_MAPPERS/microdados/dados/uberlandia_em_func_google.shp', driver='ESRI Shapefile')

# Etapa 7: Crie um mapa usando o Folium e adicione marcadores para as localizações dos prédios
# Substitua 'latitude' e 'longitude' pelas coordenadas do centro do mapa desejado
# No caso de Uberlândia, as coordenadas são: -18.9186, -48.2772
centro_mapa = (-18.9186, -48.2772)
m = folium.Map(location=centro_mapa, zoom_start=13)

print(gdf.head(40))

# Percorra cada linha no GeoDataFrame e adicione um marcador para cada prédio
for idx, row in gdf.iterrows():
    if not pd.isnull(row['geometry']):
        folium.Marker(location=[row.geometry.y, row.geometry.x], popup=row['NO_ENTIDADE']).add_to(m)

# Etapa 8: Salve o mapa como um arquivo HTML ou exiba-o em um Jupyter Notebook
m.save('E:/GitHub/UFU_MAPPERS/microdados/dados/building_locations_map_google_2.html')

# O arquivo HTML resultante conterá um mapa com marcadores para as localizações dos prédios.